<a href="https://colab.research.google.com/github/Davron030901/PyTorch/blob/main/Efficient_Data_Handling_in_PyTorch_A_Deep_Dive_into_Datasets_%26_Dataloaders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
digits = load_digits()
X,y = digits.data, digits.target

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=11)

In [ ]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
class CustomDataset(Dataset):
  def __init__(self,data,target):
    self.data=torch.tensor(data,dtype=torch.float32)
    self.target=torch.tensor(target,dtype=torch.long)

  def __len__(self):
    return len(self.data)

  def __getitem__(self,index):
    sample={'data':self.data[index],'target':self.target[index]}
    return sample

In [ ]:
train_dataset=CustomDataset(X_train,y_train)
test_dataset=CustomDataset(X_test,y_test)

In [ ]:
len(train_dataset),len(test_dataset)

(1437, 360)

In [ ]:
train_dataset[0]

{'data': tensor([ 0.0000,  0.7855,  1.4473,  0.9870,  0.9711,  1.6287, -0.4107, -0.1187,
         -0.0558,  1.5645,  0.4803, -1.2699, -0.4852,  1.3371, -0.5054, -0.1243,
         -0.0354, -0.7298, -1.5627, -1.2109,  0.1465,  1.0232, -0.5341, -0.1049,
         -0.0373, -0.7800, -0.3372,  1.0441,  0.9914,  1.4639,  2.3861, -0.0264,
          0.0000,  0.1808,  1.1503,  0.4545,  0.7962, -0.8123, -0.2567,  0.0000,
         -0.0577, -0.5369, -0.9146,  0.7433, -0.0886, -1.4491, -0.8084, -0.0940,
         -0.0334, -0.4056, -0.9975,  1.2375, -1.3842, -1.4744, -0.7676, -0.2048,
         -0.0264, -0.2901,  1.4802, -0.7153, -2.4151, -1.1653, -0.4980, -0.1942]),
 'target': tensor(7)}

In [ ]:
train_data_loader=DataLoader(train_dataset,batch_size=32,shuffle=True,num_workers=3)
test_data_loader=DataLoader(test_dataset,batch_size=32,shuffle=False,num_workers=3)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
class SimpleNN(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(SimpleNN,self).__init__()
    self.layer1=nn.Linear(input_size,hidden_size)
    self.relu=nn.ReLU()
    self.layer2=nn.Linear(hidden_size,output_size)

  def forward(self,x):
    x=self.layer1(x)
    x=self.relu(x)
    x=self.layer2(x)
    return x

In [ ]:
input_size=X_train.shape[1]
hidden_size=64
output_size=len(set(y_train))
model=SimpleNN(input_size,hidden_size,output_size)

In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

In [ ]:
num_epochs=10

for epoch in range(num_epochs):
  model.train()
  running_loss=0.0
  for batch in train_data_loader:
    inputs=batch['data']
    target=batch['target']
    optimizer.zero_grad()
    output=model(inputs)
    loss=criterion(output,target)
    loss.backward()
    optimizer.step()
    running_loss+=loss.item()
  print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_data_loader)}")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 1/10, Loss: 1.9330133146709867
Epoch 2/10, Loss: 1.1397264109717475
Epoch 3/10, Loss: 0.6043149749437968
Epoch 4/10, Loss: 0.36656732360521954
Epoch 5/10, Loss: 0.2588196870353487
Epoch 6/10, Loss: 0.20078441600004832
Epoch 7/10, Loss: 0.16165988677077822
Epoch 8/10, Loss: 0.1335863375829326
Epoch 9/10, Loss: 0.1138131159875128
Epoch 10/10, Loss: 0.09779247500830227


In [ ]:
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for batch in test_data_loader:
        inputs = batch['data']
        target = batch['target']
        output = model(inputs)

        # Get the predicted class indices
        _, preds = torch.max(output, dim=1)  # preds is the indices

        all_preds.extend(preds.cpu().numpy())  # Convert to numpy and extend
        all_targets.extend(target.cpu().numpy())  # Convert to numpy and extend


In [ ]:
accuracy=accuracy_score(all_targets,all_preds)
print(f"Test Accuracy: {accuracy*100:.3f}")

Test Accuracy: 97.222
